In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('sleep_health_clean.csv',sep=';')

In [2]:
data.describe()

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Systolic Blood Pressure,Diastolic Blood Pressure,Heart Rate,Daily Steps
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,187.500000,42.184492,7.132086,7.312834,59.171123,5.385027,128.553476,84.649733,70.165775,6816.844920
std,108.108742,8.673133,0.795657,1.196956,20.830804,1.774526,7.748118,6.161611,4.135676,1617.915679
min,1.000000,27.000000,5.800000,4.000000,30.000000,3.000000,115.000000,75.000000,65.000000,3000.000000
25%,94.250000,35.250000,6.400000,6.000000,45.000000,4.000000,125.000000,80.000000,68.000000,5600.000000
50%,187.500000,43.000000,7.200000,7.000000,60.000000,5.000000,130.000000,85.000000,70.000000,7000.000000
75%,280.750000,50.000000,7.800000,8.000000,75.000000,7.000000,135.000000,90.000000,72.000000,8000.000000
max,374.000000,59.000000,8.500000,9.000000,90.000000,8.000000,142.000000,95.000000,86.000000,10000.000000


In [3]:
data.columns

Index(['Person ID', 'Gender', 'Age', 'Occupation', 'Sleep Duration',
       'Quality of Sleep', 'Physical Activity Level', 'Stress Level',
       'BMI Category', 'Systolic Blood Pressure', 'Diastolic Blood Pressure',
       'Heart Rate', 'Daily Steps', 'Sleep Disorder'],
      dtype='object')

In [4]:
for col in data:
    print(col)
    print(len(data[col].unique()))
    print()

Person ID
374

Gender
2

Age
31

Occupation
11

Sleep Duration
27

Quality of Sleep
6

Physical Activity Level
16

Stress Level
6

BMI Category
4

Systolic Blood Pressure
18

Diastolic Blood Pressure
17

Heart Rate
19

Daily Steps
20

Sleep Disorder
3



In [5]:
data["Sleep Disorder"] = data["Sleep Disorder"].replace([None], "None")
data.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Systolic Blood Pressure,Diastolic Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,2,Male,28,Doctor,6.2,6,60,8,Normal,125,80,75,10000,None
1,3,Male,28,Doctor,6.2,6,60,8,Normal,125,80,75,10000,None
2,34,Male,31,Doctor,6.1,6,30,8,Normal,125,80,72,5000,None
3,36,Male,31,Doctor,6.1,6,30,8,Normal,125,80,72,5000,None
4,37,Male,31,Doctor,6.1,6,30,8,Normal,125,80,72,5000,None


In [6]:
data.drop(columns=["Person ID"],inplace=True)

In [7]:

X = data.drop(columns=["Sleep Disorder"], inplace=False)
y = data["Sleep Disorder"]

In [8]:
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'bool']).columns

from sklearn.preprocessing import OrdinalEncoder

gender_encoder = OrdinalEncoder(categories=[["Female", "Male"]], dtype=np.int8)

data["Gender"] = gender_encoder.fit_transform(data["Gender"].values.reshape(-1,1))
data.head()

bmi_encoder = OrdinalEncoder(categories=[["Normal", "Normal Weight", "Overweight", "Obese"]], dtype=np.int8)

data["BMI Category"] = bmi_encoder.fit_transform(data["BMI Category"].values.reshape(-1,1))
data.head()

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
ct = ColumnTransformer(transformers=
    [("one_hot_encoder",OneHotEncoder(),["Occupation"]),
     ("bmi_encoder",OrdinalEncoder(categories=[["Normal", "Normal Weight", "Overweight", "Obese"]], dtype=np.int8),["BMI Category"]),
     ("gender_encoder", OrdinalEncoder(categories=[["Female", "Male"]], dtype=np.int8),["Gender"])], remainder='passthrough'
)
X = ct.fit_transform(X)


In [10]:
cols = ct.get_feature_names_out()
X = pd.DataFrame(data=X, columns=cols)

In [11]:
y.shape

(374,)

In [11]:
y = y.values.reshape(-1,1)

In [12]:

y_encoder = OneHotEncoder()

y = y_encoder.fit_transform(y)



In [13]:
y = y.toarray()

In [14]:
y = pd.DataFrame(y, columns=y_encoder.get_feature_names_out())

In [17]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=101)

In [18]:
X_train.to_csv('data/X_train.csv', index_label=False)
X_test.to_csv('data/X_test.csv', index_label=False)
y_train.to_csv('data/y_train.csv', index_label=False)
y_test.to_csv('data/y_test.csv', index_label=False)